In [ ]:
import calliope
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Steps to achieve multi-objective optimization (ε-cut):
1. prepare cost data (monetary: HSLU database; emission: KBOB, limitation: different database);
2. Input into calliope configuration;
3. Define available technology;
4. Solve for min-cost $(C_L, E_L)$ and min-emission $(C_R, E_R)$;
5. define amount of cuts (n), and primary objective (normally Cost)
6. Divide emission range $[E_L, E_R]$ into n parts, $E_0 = E_L, E_1, ..., E_i, ..., E_n-1, E_n=E_R$;
7. optimize for C, with constriaint of $E\leq E_i$;
8. get n+1 points: $(C_0, E_0) = (C_L, E_L), (C_1, E_1), ..., (C_i, E_i), ..., (C_n-1, E_n-1), (C_n, E_n) = (C_R, E_R)$
9. link these points in a coordinate plane to form the pareto front.

In [ ]:
# define a function that solves for cost-emission tuple for a give emission or cost constraint
def solve_for_cost_emission_tuple(config: calliope.AttrDict, timeseries_df: dict, obj_type_now_have: str = 'monetary', obj_now_have: float = 0):
    if obj_type_now_have == 'monetary':
        # first configure the config file for min emission with system wide cost constraint
        config_min_emission: calliope.AttrDict = config.copy()
        config_min_emission.set_key(key='run.objective_options.cost_class', value={'emissions': 1})
        config_min_emission.set_key(key='group_constraints.systemwide_cost_max.cost_max.monetary', value=obj_now_have)
        

In [ ]:
# define the epsilon-cut method for pareto front
def epsilon_cut(config: [dict, calliope.AttrDict], epsilon: int, timeseries_df: dict):
    # first configure the config file for min cost and min emission
    config_min_emission = config.copy()
    calliope.AttrDict.set_key(config_min_emission, key='run.objective_options.cost_class', value={'emissions': 1})
    
    # solve for min emission
    model_min_emission = calliope.Model(config_min_emission, timeseries_dataframe=timeseries_df)
    model_min_emission.run()
    
    # update config, so that the model optimizes for min cost with the tightest emission constraint, which is to find the max cost
    model_min_emission_max_cost = model_min_emission.copy()
